In [138]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import nltk
import ast
# from curtsies.fmtfuncs import red, bold, green, on_blue, yellow

In [69]:
# read in rationales dataframe
rationales_df = pd.read_csv("./artifacts/RCNN_IB_GAN_be_mimic3_org_embs2021-05-12/batch_all_chosen_words.csv", index_col=0)
rationales_df.head()

,raw_sentence,chosen_words,words_not_chosen,predicted_label,true_label,enq_seq_chosen,enq_seq_not_chosen
0,"['date', 'of', 'birth', 'sex', 'f', 'service',...","['UNK', 'haldol', 'bp', 'UNK', 'UNK', 'UNK', '...","['date', 'of', 'birth', 'sex', 'service', 'med...",0,1,"[3, 1658, 94, 3, 3, 3, 3, 3, 3, 3, 8, 3, 3, 3,...","[115, 8, 438, 487, 211, 670, 146, 475, 278, 26..."
0,"['pneumonia', 'echo', 'the', 'left', 'atrium',...","['at', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'glu...","['pneumonia', 'echo', 'the', 'left', 'atrium',...",0,1,"[19, 3, 3, 3, 3, 3, 149, 3, 238, 3, 526, 3, 58...","[335, 329, 5, 18, 126, 14, 21, 12, 82, 18, 44,..."
0,"['may', 'recommend', 'lisinopril', 'low', 'dos...","['may', 'recommend', 'lisinopril', 'dose', 'as...","['low', 'an', 'outpatient', 'follow', 'up', 'i...",0,1,"[219, 1667, 696, 151, 22, 7, 3848, 3, 24, 442,...","[161, 70, 499, 121, 93, 12, 3, 1045, 9, 1050, ..."
0,"['should', 'develop', 'any', 'chest', 'pain', ...",['UNK'],"['should', 'develop', 'any', 'chest', 'pain', ...",0,1,[3],"[342, 2306, 185, 81, 43, 557, 8, 275, 444, 613..."
0,"['date', 'of', 'birth', 'sex', 'f', 'service',...","['UNK', 'penicillins', 'with', 'solu', 'UNK', ...","['date', 'of', 'birth', 'sex', 'service', 'med...",1,1,"[3, 1475, 9, 5506, 3, 3, 7, 2325, 3, 3, 3, 94,...","[115, 8, 438, 487, 211, 670, 146, 1460, 475, 2..."


In [73]:
# convert these weird strings of lists to lists
rationales_df['raw_sentence'] = rationales_df['raw_sentence'].apply(ast.literal_eval)
rationales_df['chosen_words'] = rationales_df['chosen_words'].apply(ast.literal_eval)
rationales_df['words_not_chosen'] = rationales_df['words_not_chosen'].apply(ast.literal_eval)
rationales_df['enq_seq_chosen'] = rationales_df['enq_seq_chosen'].apply(ast.literal_eval)
rationales_df['enq_seq_not_chosen'] = rationales_df['enq_seq_not_chosen'].apply(ast.literal_eval)

In [ ]:
def detokenize(tokens):
    """Slightly cleaner version of joining with spaces.
    Args:
        tokens (list<string>): the sentence to print
    Return:
        str: the sentence
    """
    return ''.join([
        ' ' + t if not t.startswith('\'') and
                   t not in string.punctuation
        else t
        for t in tokens]).strip().capitalize()

In [134]:
def print_formatted_words(raw_sentence,chosen_words):
    
    
    '''
    Function to combine original raw sentences with the rationales/chosen words from model
    
    args: raw_sentence - tokenized list
          chosen_words - tokenized list
          
    returns:
            a detokenized string with colour formatting added to rationale/chosen words, embedded into raw text
    '''
    
    #get the set of chosen words - do not want repeats
    st = set(chosen_words)
    

#     t = rationales_df.iloc[2]['raw_sentence']
    
    #iterate over raw sentence and get index of words that appear in chosen words
    indices = [i for i, e in enumerate(raw_sentence) if e in st]
    # print(len(indices))
    # print(indices)

    #run back over raw sentence and colourise any chosen words, then reorganise into order of original sentence
    formatted_words = []
    for i,word in enumerate(raw_sentence):
        if i in indices:
    #         print('\033[34m' + word + '\033[0m')
            formatted_words.append('\033[34m' + word + '\033[0m')

        else:
    #         print(word)
            formatted_words.append(word)
    
    formatted_sentence = detokenize(formatted_words)

    
    print(f"original sentence:\n{detokenize(raw_sentence)}")
    print("\n =====================================================================================================\n")          
    print(formatted_sentence)
    
#     return formatted_sentence





In [170]:
# print_formatted_words(rationales_df.iloc[2]['raw_sentence'], rationales_df.iloc[2]['chosen_words'])

## False Positive formatted example - Gets a random sample from dataframe

In [165]:
# false positive - pred 1 true 0 - get random sample
fp_df = rationales_df[(rationales_df["true_label"]==0) & rationales_df["predicted_label"]==1].sample().squeeze(axis = 0)


print_formatted_words(fp_df['raw_sentence'], fp_df['chosen_words'])

original sentence:
Duration of his stay medications on admission flexeril neurontin insulin discharge medications lactulose 10 gram 15 ml syrup sig forty five 45 ml po tid 3 times a day rifaximin 200 mg tablet sig two 2 tablet po tid 3 times a day thiamine hcl 100 mg tablet sig one 1 tablet po daily daily folic acid 1 mg tablet sig one 1 tablet po daily daily multivitamin tablet sig one 1 tablet po daily daily insulin lispro 100 unit ml solution sig according to sliding scale subcutaneous asdir as directed spironolactone 25 mg tablet sig two 2 tablet po daily daily furosemide 40 mg tablet sig one 1 tablet po daily daily pantoprazole 40 mg recon soln sig one 1 recon soln intravenous q12h every 12 hours discharge disposition extended care discharge diagnosis primary esophageal variceal bleed hepatic encephalopathy alcoholic hcv cirrhosis secondary alcoholism diabetes mellitus discharge condition mental status confused sometimes has not been able to state date correctly but oriented to pe

## False Negative formatted example - Gets a random sample from dataframe

In [167]:
# false negative - pred 0 true 1

fn_df = rationales_df[(rationales_df["true_label"]==1) & rationales_df["predicted_label"]==0].sample().squeeze(axis = 0)


print_formatted_words(fn_df['raw_sentence'], fn_df['chosen_words'])

original sentence:
Much improved prior to discharge please call your doctor or go to the emergency department if you experience new chest pain pressure squeezing or tightness you develop new or worsening cough shortness of breath or wheezing you are vomiting and cannot keep down fluids or your medications if you are getting dehydrated due to continued vomiting diarrhea or other reasons signs of dehydration include dry mouth rapid heartbeat or feeling dizzy or faint when standing you see blood or dark black material when you vomit or have a bowel movement you experience burning when you urinate have blood in your urine or experience an unusual discharge your pain is not improving within 12 hours or is not under control within 24 hours your pain worsens or changes location you have shaking chills or fever greater than 5 degrees fahrenheit or 38 degrees celsius you develop any other concerning symptoms changes in your medication reconciliation upon admission we added start multivitamin 1 

## True positive -  gets random sample from dataframe

In [168]:
# true positive - pred 1 true 1
tp_df = rationales_df[(rationales_df["true_label"]==1) & rationales_df["predicted_label"]==1].sample().squeeze(axis = 0)


print_formatted_words(tp_df['raw_sentence'], tp_df['chosen_words'])

original sentence:
For pt s thrombocytopenia included mds hit less likely drug related on lasix pt s anticoagulants plavix aspirin were held no evidence of dic hit antibody is pending renal failure pt was admitted in acute renal failure thought to be secondary to uric acid nephropathy less likely etiology is rhabdomyalysis peak of 1500 resolved with iv hydration and allopurinol cv no active issues pt was ruled out for mi aspirin was held in setting of low plt and increased inr pt was continued on lopressor lipitor isordil amlodipine and lisinopril leg pain initial concern for nerve compression or blastoma however pain resolved without intervention or need for imaging elevated ck unclear etiology ck peaked at ck normalized with iv hydration medications on admission cephalexin 250 mg qid asa 81 mg po qam plavix 75 mg po qd pentoxifylline 800 mg po qam imdur 120 mg po qam lasix 40 mg po bid lipitor 20 mg po qd norvasc 10 mg po qpm atenolol 100 mg po qam lisinopril 10 mg po qam combivent 2

## True negative - gets random sample from dataframe

In [169]:
# true negative - pred 0 true 0
tn_df = rationales_df[(rationales_df["true_label"]==0) & rationales_df["predicted_label"]==0].sample().squeeze(axis = 0)


print_formatted_words(tn_df['raw_sentence'], tn_df['chosen_words'])

original sentence:
Date of birth sex m service medicine allergies patient recorded as having no known allergies to drugs attending chief complaint direct admit from osh for transplant work up major surgical or invasive procedure paracentesis placement of tunnel line for hemodialysis history of present illness pt is 54 yo male with alcoholic cirrhosis h o esophageal varices who has had multiple recent admissions to southern medical center for ascites and hepatic encephalopathy pt presented recently to the osh with encephalopathy and developed oliguric arf severe hyponatremia and severe hyperkalemia he also had a transaminitis on admission to osh which improved during his hospital stay he had diarrhea and was found to be positive for c dif presumably recent abx treatment for sbp and was placed on flagyl he was treated with octreotide midodrine and albumin for likely hrs he was initially treated with levofloxacin but this was d c d after a negative urine cx pt also reportedly underwent th